# 두개의 투자전략을 클래스로 선언
1. 생성자
    - 클래스가 생성될 때 최초 한 번만 실행
    - 객체 변수(self)에 데이터프레임, 기준컬럼명, 시작시간, 종료시간

2. buyandhold 함수
    - 객체 변수를 이용한 백테스팅
    - 결과와 누적수익률 리턴

3. ballenger
    - 밴드 생성 함수
        - 상단, 하단 밴드, 이동평균선 생성
    - 거래 내역 추가 (trade)
        - 밴드 기준으로
    - 수익률 계산 함수
        - 매도 시 수익률이 발생, 누적 수익률 리턴 (데이터프레임)

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime

/var/folders/lj/ms0l79910fz22cg2hchczhw40000gn/T/ipykernel_7870/3172591061.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [46]:
class Invest :
    def __init__(self, _df, _col = 'Adj Close', _start = '2010-01-01', _end = datetime.now()) :
        flag = _df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
        self.df = _df.loc[~flag, ]

        if 'Date' in self.df.columns :
            self.df.set_index('Date', inplace = True)

        self.df.index = pd.to_datetime(self.df.index, format='%Y-%m-%d')
        self.df = self.df[[_col]] # [] : Series, [[]] : DataFrame

        self.col = _col
        try :
            self.start = datetime.strptime(_start, '%Y-%m-%d')
            if type(_end) == 'str' :
                self.end = datetime.strptime(_end, '%Y-%m-%d')
            else : 
                self.end = _end
        except :
            print('지원하지 않는 형식입니다.')

    def buyAndHold(self) :
        result = self.df.loc[self.start:self.end, ]
        result['rtn'] = (result[self.col].pct_change() + 1).fillna(1)

        result['acc_rtn'] = result['rtn'].cumprod()
        acc_rtn = result.iloc[-1]['acc_rtn']
        return result, acc_rtn
    
    def ballenger(self, _cnt = 20) :
        result = self.df.copy()
        result['center'] = result[self.col].rolling(_cnt).mean()
        result['ub'] = result['center'] + (2 * result[self.col].rolling(_cnt).std())
        result['lb'] = result['center'] - (2 * result[self.col].rolling(_cnt).std())

        result = result.loc[self.start : self.end, ]

        result['trade'] = ""
        for i in result.index :
            if result.loc[i, self.col] >= result.loc[i, 'ub'] :
                result.loc[i, 'trade'] = "" # 보유중이면 매도, 아닐경우 유지
            elif result.loc[i, self.col] <= result.loc[i, 'lb'] :
                result.loc[i, 'trade'] = "buy" # 보유중이면 유지, 아닐경우 매수
            else :
                if result.shift().loc[i, 'trade'] == '' :
                    result.loc[i, 'trade'] = result.shift().loc[i, 'trade'] # 어제값과 오늘값이 같다 -> 유지

        # 수익률 계산
        result['rtn'] = 1

        for i in result.index :
            if (result.shift().loc[i, 'trade'] == "") & (result.loc[i, 'trade'] == 'buy') :
                buyPrice = result.loc[i, self.col]
                print(f'매수일 : {i}, 매수가 : {buyPrice}')
                
            elif (result.shift().loc[i, 'trade'] == 'buy') & (result.loc[i, 'trade'] == '') :
                sellPrice = result.loc[i, self.col]
                rtn = sellPrice / buyPrice
                result.loc[i, 'rtn'] = rtn
                print(f'매도일 : {i}, 매도가 : {sellPrice}, 수익률 : {round(rtn, 2)}') # round : 소수자리 반올림, 2자리까지 출력

        result['acc_rtn'] = result['rtn'].cumprod()
        acc_rtn = result.iloc[-1]['acc_rtn']

        return result, acc_rtn
            


In [6]:
AAPL = pd.read_csv('../../csv/AAPL.csv')

In [47]:
class1 = Invest(AAPL)
class1.df

,Adj Close
Date,
1980-12-12,0.410525
1980-12-15,0.389106
1980-12-16,0.360548
1980-12-17,0.369472
1980-12-18,0.380182
...,...
2019-06-18,198.449997
2019-06-19,197.869995
2019-06-20,199.460007


In [ ]:
class1.buyAndHold()

In [ ]:
bnh_df, bnh_rtn = class1.buyAndHold()

In [10]:
bnh_rtn

7.436513727083075

In [11]:
bnh_df.tail()

,Adj Close,rtn,acc_rtn
Date,,,
2019-06-18,198.449997,1.023518,7.409631
2019-06-19,197.869995,0.997077,7.387975
2019-06-20,199.460007,1.008036,7.447342
2019-06-21,198.779999,0.996591,7.421952
2019-06-24,199.169998,1.001962,7.436514


In [21]:
class1.ballenger()['trade'].value_counts()

trade
       1426
buy     945
Name: count, dtype: int64

In [48]:
class1.ballenger()

매수일 : 2010-01-22 00:00:00, 매수가 : 24.747818
매도일 : 2010-01-25 00:00:00, 매도가 : 25.413599, 수익률 : 1.03
매수일 : 2010-01-28 00:00:00, 매수가 : 24.940546
매도일 : 2010-02-01 00:00:00, 매도가 : 24.369871, 수익률 : 0.98
매수일 : 2010-08-24 00:00:00, 매수가 : 30.026524
매도일 : 2010-08-25 00:00:00, 매도가 : 30.396961, 수익률 : 1.01
매수일 : 2011-03-16 00:00:00, 매수가 : 41.299767
매도일 : 2011-03-17 00:00:00, 매도가 : 41.879189, 수익률 : 1.01
매수일 : 2011-05-16 00:00:00, 매수가 : 41.711502
매도일 : 2011-05-17 00:00:00, 매도가 : 42.066914, 수익률 : 1.01
매수일 : 2011-06-10 00:00:00, 매수가 : 40.785408
매도일 : 2011-06-13 00:00:00, 매도가 : 40.873005, 수익률 : 1.0
매수일 : 2011-06-20 00:00:00, 매수가 : 39.461357
매도일 : 2011-06-21 00:00:00, 매도가 : 40.710316, 수익률 : 1.03
매수일 : 2011-11-14 00:00:00, 매수가 : 47.463268
매도일 : 2011-11-15 00:00:00, 매도가 : 48.660904, 수익률 : 1.03
매수일 : 2011-11-21 00:00:00, 매수가 : 46.180508
매도일 : 2011-11-22 00:00:00, 매도가 : 47.119102, 수익률 : 1.02
매수일 : 2012-04-16 00:00:00, 매수가 : 72.601524
매도일 : 2012-04-17 00:00:00, 매도가 : 76.302109, 수익률 : 1.05
매수일 : 2012-04-20 00:0

/var/folders/lj/ms0l79910fz22cg2hchczhw40000gn/T/ipykernel_7870/1852488431.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.0269026142021895' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.loc[i, 'rtn'] = rtn


매수일 : 2016-01-07 00:00:00, 매수가 : 90.743942
매도일 : 2016-01-11 00:00:00, 매도가 : 92.700867, 수익률 : 1.02
매수일 : 2016-04-27 00:00:00, 매수가 : 92.532272
매도일 : 2016-05-03 00:00:00, 매도가 : 90.034988, 수익률 : 0.97
매수일 : 2016-06-20 00:00:00, 매수가 : 90.507019
매도일 : 2016-06-21 00:00:00, 매도가 : 91.277893, 수익률 : 1.01
매수일 : 2016-06-24 00:00:00, 매수가 : 88.88913
매도일 : 2016-06-28 00:00:00, 매도가 : 89.069954, 수익률 : 1.0
매수일 : 2016-09-09 00:00:00, 매수가 : 98.680893
매도일 : 2016-09-12 00:00:00, 매도가 : 100.891243, 수익률 : 1.02
매수일 : 2016-11-01 00:00:00, 매수가 : 106.680237
매도일 : 2016-11-07 00:00:00, 매도가 : 106.189255, 수익률 : 1.0
매수일 : 2017-06-09 00:00:00, 매수가 : 144.5
매도일 : 2017-06-19 00:00:00, 매도가 : 141.939392, 수익률 : 0.98
매수일 : 2017-09-20 00:00:00, 매수가 : 151.971283
매도일 : 2017-09-26 00:00:00, 매도가 : 149.118195, 수익률 : 0.98
매수일 : 2018-01-29 00:00:00, 매수가 : 164.136932
매도일 : 2018-01-31 00:00:00, 매도가 : 163.618988, 수익률 : 1.0
매수일 : 2018-02-02 00:00:00, 매수가 : 156.846741
매도일 : 2018-02-06 00:00:00, 매도가 : 159.319138, 수익률 : 1.02
매수일 : 2018-03-21 0

(             Adj Close      center          ub          lb trade  rtn  acc_rtn
 Date                                                                          
 2010-01-04   26.782711   25.037723   27.046734   23.028713        1.0  1.00000
 2010-01-05   26.829010   25.169503   27.288098   23.050908        1.0  1.00000
 2010-01-06   26.402260   25.307290   27.366449   23.248130        1.0  1.00000
 2010-01-07   26.353460   25.436879   27.410937   23.462821        1.0  1.00000
 2010-01-08   26.528664   25.525609   27.529742   23.521475        1.0  1.00000
 ...                ...         ...         ...         ...   ...  ...      ...
 2019-06-18  198.449997  185.432500  201.032574  169.832427        1.0  1.42265
 2019-06-19  197.869995  185.996000  202.558154  169.433846        1.0  1.42265
 2019-06-20  199.460007  186.830000  204.361771  169.298229        1.0  1.42265
 2019-06-21  198.779999  187.786000  205.751400  169.820600        1.0  1.42265
 2019-06-24  199.169998  188.796000  206